<a href="https://colab.research.google.com/github/badonyi/mechanism-prediction/blob/main/mechanism_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup ------------------------------------------------------------------------
# prevent false objects in subsequent runs, but keep cached data
rm(list = ls()[!ls() %in% c('ref_set', 'priors', 'fx_raw', 'handle')])

# ANSI escape codes for text colour
RED <- '\033[31m'
GREEN <- '\033[32m'
RESET <- '\033[0m'

# canonical amino acid vectors
aa1 <- strsplit('ARNDCEQGHILKMFPSTWYV', '')[[1]]
aa3 <- c('ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLU', 'GLN', 'GLY', 'HIS', 'ILE',
         'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL')

# UniProt REST API and AlphaFold database
up_api <- paste0('https://rest.uniprot.org/uniprotkb/search?query=',
                 'reviewed:true+AND+%s+organism_id:9606&format=tsv')
af_db <- 'https://alphafold.ebi.ac.uk/files/AF-%s-F1-model_v6.pdb'

# helper functions
density_sj <- function(x) density(x, bw = 'SJ', adjust = 3, n = 1024)
bayes <- function(p, prior) (p * prior) / (p * prior + (1 - p) * (1 - prior))
redstop <- function(x) stop(RED, x, RESET, call. = FALSE)


# User options -----------------------------------------------------------------
#@markdown ### <b><font color='#85b0f5'>Specify a human gene</font></b>
#@markdown #### Gene name
gene = 'SCN9A' #@param {type:"string"}
gene <- trimws(gene)

#@markdown #### <b>or</b> UniProt accession number
id = '' #@param {type:"string"}
id <- toupper(trimws(id))

#@markdown ### <b><font color='#85b0f5'>Input variants</font></b>
#@markdown Example: `A108S, L730H, T454P`
variants = 'F216S,I234T,N395K,V400M,L834R,I859T,L869H,L869F,Q886E,R907W,Y1001C,V1002L,V1309D,V1309F,V1310F,F1460V,I1472N,I1472T,F1473V,T1475I,L1623P,A1643E,T660M,G867R,R907Q,A1143P,V1310I,C1350R,D1959A,G867D,M1639K' #@param {type:"string"}


# Check 1 - Input variants -----------------------------------------------------
dr_split <- toupper(trimws(strsplit(variants, ',')[[1]]))

# wild-type, mutant, and residue positions; used later to match input
wt <- substr(dr_split, 1, 1)
mut <- substr(dr_split, nchar(dr_split), nchar(dr_split))
dr <- as.integer(gsub('[^0-9]', '', dr_split))

if (length(dr) < 3)
  redstop('EDC calculation requires at least 3 residue positions.')

if (!all(mut %in% aa1))
  redstop('Missing or non-canonical amino acids are not supported.')


# Check 2 - UniProt accession or gene symbol -----------------------------------
if (gene == '' & id == '')
  redstop('Please provide either a UniProt ID or a gene name.')

if (gene != '') {
  # match gene symbol with UniProt accession via UniProt REST API
  up_tbl <- read.delim(sprintf(up_api, gene), sep = '\t')
  if (nrow(up_tbl) > 0) {
    up_lst <- strsplit(up_tbl$Gene.Names, ' ')
    up_lst <- up_lst[lengths(up_lst) > 0]
    up_idx <- sapply(up_lst, function(x) any(tolower(x) == tolower(gene)))
    id <- up_tbl[up_idx, ]$Entry[1]
  } else {
    redstop('Gene could not be mapped to a UniProt accession ID.')
  }
}

upreg <- '[OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]([A-Z][A-Z0-9]{2}[0-9]){1,2}'
if (!grepl(upreg, id))
  redstop('UniProt accession not supported.')


# Check 3 - Fetch molecular mechanism predictions ------------------------------
if (!exists('priors')) {
  download.file('https://osf.io/download/e6b9c/', destfile = 'priors.tsv')
  priors <- read.delim('priors.tsv', sep = '\t')
}

if (id %in% priors$uniprot_id) {
  prior_set <- priors[priors$uniprot_id == id, ]
  pLOF <- prior_set$pLOF
  pDN  <- prior_set$pDN
  pGOF <- prior_set$pGOF
} else {
  redstop('UniProt accession not supported.')
}


# Fetch AlphaFold predicted model ----------------------------------------------
cat('INFO     Using the AlphaFold model',
    paste0('(', sprintf(af_db, id), ')'))

tryCatch({
  # attempt to parse ATOM lines
  a <- grep('^ATOM', readLines(sprintf(af_db, id), warn = FALSE), value = TRUE)
}, warning = function(w) {
  redstop('Structure may not exist for the given UniProt ID.')
})


# Parse alpha carbon coordinates and pLDDT from b-factor column ----------------
a <- a[trimws(substr(a, 14, 15)) == 'CA']
xyz <- data.frame(
  r = as.integer(substr(a, 23, 26)),
  x = as.numeric(substr(a, 31, 38)),
  y = as.numeric(substr(a, 39, 46)),
  z = as.numeric(substr(a, 47, 54)),
  b = as.numeric(substr(a, 61, 66))
)


# Extract sequence from structure ----------------------------------------------
uniqres <- trimws(substr(a, 18, 21))
pdb_seq <- setNames(aa1, aa3)[uniqres]
names(pdb_seq) <- xyz$r # maps relative to PDB numbering
wt_in_sturcture <- sapply(dr, function(i) pdb_seq[i])


# Process sturcture for input residue context ----------------------------------
dr_remaining <- xyz[xyz$r %in% dr, ]$r
xyz_missing <- sort(unique(dr[!dr %in% dr_remaining]))

if (length(xyz_missing) > 0)
  cat(RED, '\nWARNING  Residue(s) not found in structure:',
      paste(xyz_missing, collapse = ', '), RESET)

# filter above the pLDDT threshold
xyz <- xyz[xyz$b >= 70, ]

# disease and non-disease residue subsets
dc <- xyz[ xyz$r %in% dr, c('x','y','z')]
ac <- xyz[!xyz$r %in% dr, c('x','y','z')]

# residues removed by the pLDDT filter
dr_remaining <- xyz[xyz$r %in% dr, ]$r
n_res <- length(dr_remaining) # required for mLOF
plddt_remove <- sort(unique(dr[!dr %in% dr_remaining]))
plddt_remove <- plddt_remove[!plddt_remove %in% xyz_missing]

if (length(plddt_remove) > 0)
  cat(RED, '\nWARNING  Residue(s)', paste(plddt_remove, collapse = ', '),
      'are below the pLDDT cutoff for EDC calculation.', RESET)

if (length(dr_remaining) < 3)
  redstop('Not enough residues to calculate EDC.')


# Report any mismatched amino acids --------------------------------------------
aa_mismatch <- data.frame(
  input = as.character(dr_split[wt != wt_in_sturcture]),
  structure = as.character(wt_in_sturcture[wt != wt_in_sturcture])
)
aa_mismatch <- aa_mismatch[order(dr), ]
aa_mismatch <- aa_mismatch[!is.na(aa_mismatch$input), ]

if (nrow(aa_mismatch) > 0) {
  cat(RED, '\nWARNING  Residue(s) in structure different from input:\n\n')
  print(aa_mismatch, row.names = FALSE)
  cat(RESET)
}


# Calculate EDC ----------------------------------------------------------------
cat('\nINFO     Calculating EDC... ')
a_mat <- {
  # pairwise distances between dc and ac using matrix arithmetics
  a <- as.matrix(ac)
  b <- as.matrix(dc)
  an <- apply(a, 1, function(x) crossprod(x, x))
  bn <- apply(b, 1, function(x) crossprod(x, x))
  nn <- outer(an, bn, '+')
  sqrt(nn - 2 * tcrossprod(a, b))
}

# minimum distances for non-disease subset
a_min <- apply(a_mat, 1, min)

# pairwise and minimum distances for disease subset
d_mat <- as.matrix(dist(dc))
d_min <- apply(d_mat, 1, function(row) min(row[row != 0]))

# EDC value
edc <- mean(log(a_min)) / mean(log(d_min))

if (is.na(edc))
  redstop('EDC could not be calculated.')

cat('DONE')


# Fetch ΔΔG ranks --------------------------------------------------------------
cat('\nINFO     Retrieving ΔΔGrank scores... ')
if (!exists('fx_raw') || handle != priors[priors$uniprot_id == id, ]$handle) {
  # get OSF handle for UniProt ID and download file
  handle <- priors[priors$uniprot_id == id, ]$handle
  download.file(paste0('https://osf.io/download/', handle), destfile = 'fx.tsv')
  fx_raw <- read.delim(file = 'fx.tsv', sep = '\t')
}
cat('DONE')

# parse out ΔΔG rank values into a named vector
fx <- fx_raw[fx_raw$uniprot_id == id, ]$ddg_rank
fx <- strsplit(fx, '\\|')[[1]]
vars <- as.numeric(sub('^[^,]*,', '', fx))
names(vars) <- sub(',.*', '', fx)

# compute ΔΔG rank for valid input variants
ddg_rank <- vars[paste0(wt, dr, mut)]
n_var <- length(ddg_rank) # required for mLOF
ddg_rank <- mean(vars[paste0(wt, dr, mut)], na.rm = TRUE)

if (is.na(ddg_rank))
  redstop('ΔΔGrank could not be calculated.')


# Compute mLOF -----------------------------------------------------------------
if (!exists('ref_set')) {
  download.file('https://osf.io/download/7vgh4/', destfile = 'ref_set.tsv')
  ref_set <- read.delim('ref_set.tsv', sep = '\t')
}

# empirical EDC and ΔΔG rank distributions for LOF and non-LOF genes
lof_edc <- ref_set[ref_set$mechanism == 'LOF', ]$edc
non_lof_edc <- ref_set[ref_set$mechanism == 'non-LOF', ]$edc
lof_ddg <- ref_set[ref_set$mechanism == 'LOF', ]$ddg_rank
non_lof_ddg <- ref_set[ref_set$mechanism == 'non-LOF', ]$ddg_rank

# kernel density estimates
lof_edc_d <- density_sj(lof_edc)
non_lof_edc_d <- density_sj(non_lof_edc)
lof_ddg_d <- density_sj(lof_ddg)
non_lof_ddg_d <- density_sj(non_lof_ddg)

# probability caps
lof_edc_cap <- quantile(lof_edc, 0.1)
non_lof_edc_cap <- quantile(non_lof_edc, 0.9)
lof_ddg_cap <- quantile(lof_ddg, 0.9)
nonlof_ddg_cap <- quantile(non_lof_ddg, 0.1)

# marginal probabilities
p_lof_edc <- {
  # cap scenario
  capped_edc <- edc
  capped_edc[capped_edc > non_lof_edc_cap] <- non_lof_edc_cap
  capped_edc[capped_edc < lof_edc_cap] <- lof_edc_cap

  # indices of minimum values
  lof_idx <- sapply(
    X = capped_edc,
    FUN = function(observation) {
      which.min(abs(lof_edc_d$x - observation))
    }
  )

  nonlof_idx <- sapply(
    X = capped_edc,
    FUN = function(observation) {
      which.min(abs(non_lof_edc_d$x - observation))
    }
  )

  # density values corresponding to those indices
  density_value_nonlof <- non_lof_edc_d$y[nonlof_idx]
  density_value_lof <- lof_edc_d$y[lof_idx]

  # LOF (EDC) probability
  density_value_lof / (density_value_lof + density_value_nonlof)
}

p_lof_ddg <- {
  # cap scenario
  capped_ddg_rank <- ddg_rank
  capped_ddg_rank[capped_ddg_rank < nonlof_ddg_cap] <- nonlof_ddg_cap
  capped_ddg_rank[capped_ddg_rank > lof_ddg_cap] <- lof_ddg_cap

  # indices of minimum values
  lof_idx <- sapply(
    X = capped_ddg_rank,
    FUN = function(observation) {
      which.min(abs(lof_ddg_d$x - observation))
    }
  )

  nonlof_idx <- sapply(
    X = capped_ddg_rank,
    FUN = function(observation) {
      which.min(abs(non_lof_ddg_d$x - observation))
    }
  )

  # density values corresponding to those indices
  density_value_nonlof <- non_lof_ddg_d$y[nonlof_idx]
  density_value_lof <- lof_ddg_d$y[lof_idx]

  # LOF (ddG) probability
  density_value_lof / (density_value_lof + density_value_nonlof)
}


# missense LOF likelihood (unadjusted)
mLOF <- (p_lof_edc * n_var + p_lof_ddg * n_res) / (n_var + n_res)


# Calculate mLOF and posterior likelihoods -------------------------------------
mLOF <- (p_lof_edc * n_var + p_lof_ddg * n_res) / (n_var + n_res)
postDN <- bayes(p = 1 - mLOF, prior = pDN)
postGOF <- bayes(p = 1 - mLOF, prior = pGOF)
postLOF <- bayes(p = mLOF, prior = pLOF)


# Print metric tables ----------------------------------------------------------
knitr::kable(
  x = c(
    'EDC' = edc,
    'ΔΔGrank' = ddg_rank,
    'mLOF' = mLOF
  ),
  digits = 3,
  col.names = c('Structure-based metrics', ''),
  format = 'simple'
)

knitr::kable(
  x = c(
    'pLOF' = pLOF,
    'pGOF' = pGOF,
    'pDN' = pDN
  ),
  digits = 3,
  col.names = c('Prior mechanism propensities', ''),
  format = 'simple'
)

knitr::kable(
  x = sort(
    c(
      'postDN' = postDN,
      'postGOF' = postGOF,
      'postLOF' = postLOF
    ),
    decreasing = TRUE
  ),
  digits = 3,
  col.names = c('Ranked mechanism-specific posteriors', ''),
  format = 'simple'
)


# Provide a verdict ------------------------------------------------------------
suggestion <- c(
  'dominant-negative', 'gain-of-function', 'loss-of-function'
)[which(c(postDN, postGOF, postLOF) == pmax(postDN, postGOF, postLOF))][1]

if (suggestion != 'loss-of-function' & mLOF > 0.508) {
  cat('\n\nVERDICT', GREEN,
      'While the structural properties of the variants are more consistent',
      'with a LOF effect,\n         based upon the',
      'properties of the gene, a', suggestion, 'mechanism is expected.', RESET)
} else if (suggestion == 'loss-of-function' & mLOF < 0.508) {
  cat('\n\nVERDICT', GREEN,
      'While the structural properties of the variants are more consistent',
      'with a non-LOF effect,\n         based upon the',
      'properties of the gene, a', suggestion, 'mechanism is expected.', RESET)
} else {
  cat('\n\nVERDICT', GREEN,
      'A', suggestion, 'mechanism is most likely for the variants.', RESET)
}
cat('\n\n')

# Generate plots ---------------------------------------------------------------
options(repr.plot.width = 16, repr.plot.height = 4.5)
par(mfrow = c(1, 3), mar = c(5, 4, 4, 2) + 0.25)

# mechanism prior plot
prior_col <- c('dominant-negative' = 'pDN',
               'gain-of-function' = 'pGOF',
               'loss-of-function' = 'pLOF')[suggestion]
all_priors <- sort(priors[[prior_col]])
obs_prior <- mget(prior_col)
prior_rnk <- which(all_priors == obs_prior)[1]
prior_pctl <- sprintf('%.1fth percentile', prior_rnk / length(all_priors) * 100)

plot(all_priors,
     main = 'Observed prior mechanism propensity\nfor the suggested mechanism',
     type = 'l',
     xlab = 'All human genes',
     ylab = prior_col,
     ylim = c(0, 1),
     col = 'grey50',
     lwd = 3,
     cex.main = 1.8,
     cex.lab = 1.8,
     cex.axis = 1.8)
points(x = prior_rnk,
       y = obs_prior,
       col = 'firebrick',
       pch = 19,
       cex = 2)
text(x = 7000,
     y = 1,
     labels = prior_pctl,
     col = 'firebrick',
     cex = 1.75,
     adj = c(0, 1))

# EDC density plot
plot(lof_edc_d,
     main = 'Probability density of EDC',
     xlab = 'EDC',
     ylab = 'Density',
     col = '#fb8072',
     lwd = 3,
     xlim = c(0.5, 2.5),
     ylim = c(0, 2),
     cex.main = 1.8,
     cex.lab = 1.8,
     cex.axis = 1.8)
lines(non_lof_edc_d, lwd = 3, col = '#80b1d3')
abline(v = edc, col = '#1A1A1A80', lwd = 2, lty = 2)
legend(x = 1.5, y = 2,
       legend = c('LOF', 'non-LOF'),
       col = c('#fb8072', '#80b1d3'),
       lwd = 3,
       lty = 1,
       cex = 1.75,
       seg.len = 0.8,
       x.intersp = 0.5,
       y.intersp = 1.5,
       bty = 'n')

# ΔΔG rank density plot
plot(lof_ddg_d,
     main = 'Probability density of ΔΔGrank',
     xlab = bquote(ΔΔG[rank]),
     ylab = 'Density',
     col = '#fb8072',
     lwd = 3,
     xlim = c(0, 1),
     ylim = c(0, 3),
     cex.main = 1.8,
     cex.lab = 1.8,
     cex.axis = 1.8)
lines(non_lof_ddg_d, lwd = 3, col = '#80b1d3')
abline(v = ddg_rank, col = '#1A1A1A80', lwd = 2, lty = 2)
legend(x = -0.1, y = 3,
       legend = c('LOF', 'non-LOF'),
       col = c('#fb8072', '#80b1d3'),
       lwd = 3,
       lty = 1,
       cex = 1.75,
       seg.len = 0.8,
       x.intersp = 0.5,
       y.intersp = 1.5,
       bty = 'n')